## Getting started
This notebook looks at the BESS data from M5bat.

The data is available in a [zip file here](https://publications.rwth-aachen.de/record/985923/files/M5BAT_04-2023_RAW.zip). There's an [overview of the data here](https://publications.rwth-aachen.de/record/985923/files/Report_04-2023.pdf).

After you download the data, unzip it in your working directory. You might need to adjust the paths below.

You'll need python (3.9 or newer should be fine). And you'll need the numpy, pandas, and plotly packages, which can be `pip install`ed.

In [2]:
from datetime import datetime
import pandas as pd
import plotly.express as px

pd.options.display.float_format = '{:.1f}'.format

## BESS data
This is the data captured where the battery system connects to the grid.

Refer to https://publications.rwth-aachen.de/record/985923/files/Report_04-2023.pdf for details.

Description of the data:

| Variable | Description | Unit |
| ---- | ---- | ---- |
| DateAndTime | Date and Time | UTC Timezone ('yyyy-MM-dd HH:mm:ss')|
| M5BAT_P | Active power of M5BAT measured at the network node | kW (- = charging; + = discharging) |
| M5BAT_Q | Reactive power of M5BAT measured at the network node | kVAr |
| Grid_frequency | Grid frequency measured at the network node | mHz |
| Temperature | Ambient temperature at M5BAT site | 0.1°C |
| FCR_activated | Activation signal for FCR | True = FCR activated |
| FCR_P | Active Power for FCR (calculation) | kW (- = charging; + = discharging) | 
| FCR_control | Control band for FCR | kW |
| SPA_ask_P | Request for active power for setpoint adjustment | kW (- = charging; + = discharging) |
| SPA_exec_P | Active power for setpoint adjustment | kW (- = charging; + = discharging) |
| SOC | State of Charge for M5BAT (calculated) | %|
| Interpolated | Interpolation signal for data evaluation | True = Value linear interpolated|

In [3]:
# Read the BESS data. If you are having trouble with the raw data, go to the next cell for a work-around.
# This is the grid connection data.
bess = pd.read_csv('BESS.csv', sep=';', parse_dates=['DateAndTime'], index_col='DateAndTime')

# We'll downsample this massively as we do an initial exploration.
bess = bess.sample(frac=.001, random_state=13).sort_index()  # data gets scrambled when sampling, so we re-sort
bess

,M5BAT_P,M5BAT_Q,Grid_frequency,Temperature,FCR_activated,FCR_P,FCR_control,SPA_ask_P,SPA_exec_P,SOC,interpolated
DateAndTime,,,,,,,,,,,
2023-04-01 00:01:22,484,21,49967,91,1,495,3000,0,0,37,0
2023-04-01 00:24:15,-2,-3,50002,92,1,0,3000,0,0,36,0
2023-04-01 00:50:59,-223,-6,50012,90,1,-210,3000,0,0,37,0
2023-04-01 01:00:20,1,21,49999,90,1,0,3000,0,0,37,0
2023-04-01 01:33:07,5,0,49999,90,1,0,3000,0,0,37,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 22:13:01,-199,1,50009,116,1,-165,3000,0,0,35,0
2023-04-30 23:10:07,-2,-3,50004,109,1,0,3000,-250,0,34,0
2023-04-30 23:21:21,-252,-4,50001,110,1,0,3000,-650,-248,35,0


In [ ]:
# Alternatively, use the pre-downsampled file that Lawrence sent out
bess = pd.read_parquet('bess.parquet')
bess

In [4]:
# First, summarize the data
bess.describe()

,M5BAT_P,M5BAT_Q,Grid_frequency,Temperature,FCR_activated,FCR_P,FCR_control,SPA_ask_P,SPA_exec_P,SOC,interpolated
count,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0,2592.0
mean,-54.6,0.2,49982.1,91.5,1.0,-17.2,2973.4,-30.4,-33.8,46.6,0.0
std,409.0,21.4,982.5,37.6,0.1,378.6,281.4,177.0,177.4,10.5,0.0
min,-1735.0,-204.0,0.0,-8.0,0.0,-1455.0,0.0,-750.0,-748.0,0.0,0.0
25%,-283.2,-7.0,49985.0,69.0,1.0,-255.0,3000.0,0.0,0.0,38.0,0.0
50%,-3.0,-2.0,50002.0,90.0,1.0,0.0,3000.0,0.0,0.0,44.0,0.0
75%,199.0,10.0,50017.0,114.0,1.0,225.0,3000.0,0.0,0.0,52.0,0.0
max,2101.0,102.0,50108.0,214.0,1.0,1500.0,3000.0,1500.0,1488.0,76.0,0.0


In [5]:
# The basic job of the battery system is to maintain grid frequency, so let's start with that column.
# We can see it's generally arond 50k (==50hz), but it looks like there are some 0s that are probably
# junk. So if we plot it, it'll be hard to read:

px.line(bess, x=bess.index, y='Grid_frequency')

In [6]:
# You can zoom in manually, or just drop the 0s for now
tp = bess.loc[bess.Grid_frequency > 0]
px.line(tp, x=tp.index, y='Grid_frequency')  # note we can omit the x here; it will default to the index

So generally the frequency is 50hz +/- .1hz, though keep in mind we heavily downsampled, so might be losing
larger excursions.

In [8]:
px.line(bess, y='Temperature')

In [9]:
# We see a daily rhythm to the temperature. The units here are .1°C, so the range is roughly 30-70°F.

# Let's also look at how these temperatures are distributed.
px.histogram(bess, x='Temperature')

The mode is in the 8°-10°C range. Temperature has an impact on battery degradation, so it'll be useful to keep this range in mind.

In [11]:
# Finally let's look at the power of the system

In [12]:
px.line(bess, y='M5BAT_P')

In [ ]:
# This feels familiar... 
#
# At a high level, the system should be absorbing energy when the frequency is too high, and releasing energy when it is too low.
#
# Can we see that behavior? Let's compare the active power to the grid frequency we looked at earlier.

In [70]:
tp = tp.loc[tp.Grid_frequency > 0]
px.scatter(tp, x='Grid_frequency', y='M5BAT_P')


In [74]:
# Indeed, they are closely related: when the frequency is low, the power is positive (battery discharging);
# when it's high, the power is negative (battery charging).

In [75]:
# We can also compute the correlation:
tp = tp.loc[tp.Grid_frequency > 0]
tp.Grid_frequency.corr(tp.M5BAT_P)

-0.8726491917654134

In [ ]:
# .. which shows a strong inverse correlation.

## Exercises

1. Look at the State Of Charge (SOC) of the batteries. How does this vary? How is it related the grid frequency?
2. We looked at the *active* power above. Does the *reactive* power look similar?
